In [1]:
# Ejecucion de queries
from helper.helper import Helper

# Funciones adicionales
import sys
import os
sys.path.insert(1, '../utils')
from replica_n_meses.replica_n_meses import replica
from tendencias.tendencias import tendencias

In [6]:
helper = Helper(username='warboled', dsn='impala-virtual-prd')

general_params = {
    'proccess_zone': 'proceso',
    'proccess_area_zone': 'proceso_cap_analit_y_gob_de_inf',
    'pref': 'pmod'
}

join = """
    t1.nit_enmascarado = t2.nit_enmascarado
    AND t1.num_oblig_enmascarado = t2.num_oblig_enmascarado
    AND t1.num_oblig_orig_enmascarado = t2.num_oblig_orig_enmascarado
    AND t1.fecha_var_rpta_alt = t2.fecha_var_rpta_alt
"""
vars_agrupa = ['nit_enmascarado', 'num_oblig_enmascarado', 'num_oblig_orig_enmascarado', 'fecha_var_rpta_alt']

path_etl = os.path.abspath(os.getcwd()).replace('notebooks', "etl")
path_tend = os.path.abspath(os.getcwd()).replace('notebooks', "utils/tendencias")

2024-06-23 14:35:28 - [WARNING] - No se encontro la carpeta "c:\Banco\Prueba Modelos\prueba-modelos\notebooks\logs" para guardar los logs


 ___ __  __ ____   _    _        _    
|_ _|  \/  |  _ \ / \  | |      / \   
 | || |\/| | |_) / _ \ | |     / _ \  
 | || |  | |  __/ ___ \| |___ / ___ \ 
|___|_|  |_|_| /_/   \_\_____/_/   \_\
                                      
 _   _ _____ _     ____  _____ ____  
| | | | ____| |   |  _ \| ____|  _ \ 
| |_| |  _| | |   | |_) |  _| | |_) |
|  _  | |___| |___|  __/| |___|  _ < 
|_| |_|_____|_____|_|   |_____|_| \_\
                                     



## Base Obligaciones

Crear base de abligaciones

In [7]:
helper.ejecutar_archivo(path_etl + "/0_obligaciones/0_obligaciones.sql", params=general_params)

--------------------------------------------------------------------------------------
                                  0_obligaciones.sql                                  
--------------------------------------------------------------------------------------
   i    tipo               nombre                 estado     hora_inicio   duracion   
--------------------------------------------------------------------------------------
 13/15    DROP       proceso.pmod_obligaciones   finalizado   10:55:21 AM     00:00.1 
 14/15  CREATE       proceso.pmod_obligaciones   finalizado   10:55:21 AM     00:01.5 
 15/15 COMPUTE       proceso.pmod_obligaciones   finalizado   10:55:23 AM     00:01.0 
--------------------------------------------------------------------------------------

Resumen de consultas:
---------------------------------------------
       documento          total    duracion 
---------------------------------------------
       0_obligaciones.sql      9     00:10.7 
 1_obligaci

## Replica Clientes

Replica n meses de las obligaciones para construcción de variables tendencia

In [10]:
params_replica = general_params
params_replica['base'] = 'pmod_obligaciones'
params_replica['fecha_corte'] = 'fecha_var_rpta_alt'
params_replica['vars_base'] = ",".join(["nit_enmascarado", "num_oblig_enmascarado", "num_oblig_orig_enmascarado"])
params_replica['path'] = os.path.abspath(os.getcwd()).replace('notebooks', "utils\\replica_n_meses")

In [11]:
replica(helper, 202308, 202401, params_replica)

Fechas del intervalo:  202208 202401
------------------------------------------------------------------------------
                              replica_fechas.sql                              
------------------------------------------------------------------------------
  i   tipo            nombre              estado     hora_inicio   duracion   
------------------------------------------------------------------------------
 4/9    DROP  proceso.pmod_fechas_anio   finalizado   11:04:14 PM     00:00.0 
 5/9  CREATE  proceso.pmod_fechas_anio   finalizado   11:04:15 PM     00:00.2 
 6/9 COMPUTE  proceso.pmod_fechas_anio   finalizado   11:04:15 PM     00:00.2 
 7/9    DROP   proceso.pmod_fechas_mes   finalizado   11:04:15 PM     00:00.1 
 8/9  CREATE   proceso.pmod_fechas_mes   finalizado   11:04:15 PM     00:00.2 
 9/9 COMPUTE   proceso.pmod_fechas_mes   finalizado   11:04:15 PM     00:00.2 
------------------------------------------------------------------------------

Resumen de con

## Modulos ETL

### Base cruce Obligaciones

In [22]:
helper.ejecutar_archivo(path_etl + "/0_obligaciones/1_obligaciones_cruce.sql", params=general_params)

----------------------------------------------------------------------------------------------------
                                      1_obligaciones_cruce.sql                                      
----------------------------------------------------------------------------------------------------
    i       tipo                     nombre                     estado     hora_inicio   duracion   
----------------------------------------------------------------------------------------------------
 424/426       DROP          proceso.pmod_obligaciones_cruce   finalizado   03:40:22 PM     00:00.3 
 425/426     CREATE          proceso.pmod_obligaciones_cruce   finalizado   03:40:22 PM     00:01.9 
 426/426    COMPUTE          proceso.pmod_obligaciones_cruce   finalizado   03:40:24 PM     00:00.8 
----------------------------------------------------------------------------------------------------

Resumen de consultas:
-------------------------------------------------
         documento

### Saldos diario

In [23]:
helper.ejecutar_archivo(path_etl + "/1_saldo_diario/0_acumulado_dia.sql", params=general_params)
helper.ejecutar_archivo(path_etl + "/1_saldo_diario/1_cruce_obligaciones_sd.sql", params=general_params)

----------------------------------------------------------------------------------------------------
                                        0_acumulado_dia.sql                                         
----------------------------------------------------------------------------------------------------
    i       tipo                     nombre                     estado     hora_inicio   duracion   
----------------------------------------------------------------------------------------------------
 427/429       DROP        proceso.pmod_saldo_diario_add_dia   finalizado   03:40:28 PM     00:00.3 
 428/429     CREATE        proceso.pmod_saldo_diario_add_dia   finalizado   03:40:28 PM     00:09.3 
 429/429    COMPUTE        proceso.pmod_saldo_diario_add_dia   finalizado   03:40:38 PM     00:01.7 
----------------------------------------------------------------------------------------------------

Resumen de consultas:
-------------------------------------------------
         documento

In [24]:
params_tend = general_params

vars_tend = [
    "avg_sld_cap_final",
    "max_dia_sld_cap_final",
    "min_sld_cap_final",
    "avg_nueva_altura_mora",
    "max_nueva_altura_mora",
    "min_nueva_altura_mora",
    "avg_vlr_obligacion",
    "max_vlr_obligacion",
    "min_vlr_obligacion",
    "avg_vlr_vencido",
    "max_vlr_vencido",
    "min_vlr_vencido"
]

params_tend["table_vars"] = "proceso.pmod_saldo_diario_obligacion"
params_tend["join"] = join
params_tend["base"] = 'sd'
params_tend["vars_agrupa"] = ",".join(vars_agrupa)

In [25]:
tendencias(helper, vars_tend, path_tend, params_tend)

----------------------------------------------------------------------------------------------------
                                       0_pmod_tabla_base.sql                                        
----------------------------------------------------------------------------------------------------
    i       tipo                     nombre                     estado     hora_inicio   duracion   
----------------------------------------------------------------------------------------------------
 433/435       DROP             proceso.pmod_tend_tabla_base   finalizado   03:42:26 PM     00:00.3 
 434/435     CREATE             proceso.pmod_tend_tabla_base   finalizado   03:42:26 PM     00:01.1 
 435/435    COMPUTE             proceso.pmod_tend_tabla_base   finalizado   03:42:27 PM     00:00.8 
----------------------------------------------------------------------------------------------------

Resumen de consultas:
-------------------------------------------------
         documento

### Maestro Cuotas

In [32]:
helper.ejecutar_archivo(path_etl + "/2_maestro_cuotas/0_filtro_maestro.sql", params=general_params)
helper.ejecutar_archivo(path_etl + "/2_maestro_cuotas/1_cruce_obligaciones_mc.sql", params=general_params)

----------------------------------------------------------------------------------------------------
                                        0_filtro_maestro.sql                                        
----------------------------------------------------------------------------------------------------
    i       tipo                     nombre                     estado     hora_inicio   duracion   
----------------------------------------------------------------------------------------------------
 796/798       DROP                   proceso.maestro_cuotas   finalizado   04:25:25 PM     00:00.3 
 797/798     CREATE                   proceso.maestro_cuotas   finalizado   04:25:25 PM     00:02.2 
 798/798    COMPUTE                   proceso.maestro_cuotas   finalizado   04:25:27 PM     00:01.5 
----------------------------------------------------------------------------------------------------

Resumen de consultas:
-------------------------------------------------
         documento

In [11]:
params_tend = general_params

vars_tend = [
    "valor_cuota_mes",
    "pago_total",
    "porc_pago"
]

params_tend["table_vars"] = "proceso.pmod_maestro_cuotas_obligaciones"
params_tend["join"] = join
params_tend["base"] = 'mc'
params_tend["vars_agrupa"] = ",".join(vars_agrupa)

In [12]:
tendencias(helper, vars_tend, path_tend, params_tend)

--------------------------------------------------------------------------------------------------
                                      0_pmod_tabla_base.sql                                       
--------------------------------------------------------------------------------------------------
   i      tipo                     nombre                     estado     hora_inicio   duracion   
--------------------------------------------------------------------------------------------------
 95/97       DROP             proceso.pmod_tend_tabla_base   finalizado   02:46:48 PM     00:00.2 
 96/97     CREATE             proceso.pmod_tend_tabla_base   finalizado   02:46:48 PM     00:01.0 
 97/97    COMPUTE             proceso.pmod_tend_tabla_base   finalizado   02:46:49 PM     00:02.1 
--------------------------------------------------------------------------------------------------

Resumen de consultas:
-------------------------------------------
      documento         total    duracion 

### Master Customer Data

In [13]:
helper.ejecutar_archivo(path_etl + "/3_mcd/0_filtro_mcd.sql", params=general_params)
helper.ejecutar_archivo(path_etl + "/3_mcd/1_cruce_obligaciones_mcd.sql", params=general_params)

----------------------------------------------------------------------------------------------------
                                          0_filtro_mcd.sql                                          
----------------------------------------------------------------------------------------------------
    i       tipo                     nombre                     estado     hora_inicio   duracion   
----------------------------------------------------------------------------------------------------
 189/191       DROP                         proceso.pmod_mcd   finalizado   02:52:08 PM     00:00.2 
 190/191     CREATE                         proceso.pmod_mcd   finalizado   02:52:08 PM     00:03.9 
 191/191    COMPUTE                         proceso.pmod_mcd   finalizado   02:52:12 PM     00:01.5 
----------------------------------------------------------------------------------------------------

Resumen de consultas:
-------------------------------------------
      documento         

In [14]:
params_tend = general_params

vars_tend = [
    "total_ing",
    "tot_activos",
    "egresos_mes",
    "tot_patrimonio"
]

params_tend["table_vars"] = "proceso.pmod_mcd_obligaciones"
params_tend["join"] = join
params_tend["base"] = 'mcd'
params_tend["vars_agrupa"] = ",".join(vars_agrupa)

In [15]:
tendencias(helper, vars_tend, path_tend, params_tend)

----------------------------------------------------------------------------------------------------
                                       0_pmod_tabla_base.sql                                        
----------------------------------------------------------------------------------------------------
    i       tipo                     nombre                     estado     hora_inicio   duracion   
----------------------------------------------------------------------------------------------------
 195/197       DROP             proceso.pmod_tend_tabla_base   finalizado   02:55:03 PM     00:00.2 
 196/197     CREATE             proceso.pmod_tend_tabla_base   finalizado   02:55:03 PM     00:03.7 
 197/197    COMPUTE             proceso.pmod_tend_tabla_base   finalizado   02:55:07 PM     00:01.2 
----------------------------------------------------------------------------------------------------

Resumen de consultas:
-------------------------------------------------
         documento

### Probabilidades

In [18]:
helper.ejecutar_archivo(path_etl + "/4_pd/0_cruce_obligaciones_pd.sql", params=general_params)

----------------------------------------------------------------------------------------------------
                                    0_cruce_obligaciones_pd.sql                                     
----------------------------------------------------------------------------------------------------
    i       tipo                     nombre                     estado     hora_inicio   duracion   
----------------------------------------------------------------------------------------------------
 321/323       DROP             proceso.pmod_pd_obligaciones   finalizado   03:04:45 PM     00:00.2 
 322/323     CREATE             proceso.pmod_pd_obligaciones   finalizado   03:04:45 PM     00:06.2 
 323/323    COMPUTE             proceso.pmod_pd_obligaciones   finalizado   03:04:51 PM     00:02.3 
----------------------------------------------------------------------------------------------------

Resumen de consultas:
-------------------------------------------------
         documento

In [19]:
params_tend = general_params

vars_tend = [
    "prob_propension",
    "prob_alrt_temprana",
    "prob_auto_cura"
]

params_tend["table_vars"] = "proceso.pmod_pd_obligaciones"
params_tend["join"] = join
params_tend["base"] = 'pd'
params_tend["vars_agrupa"] = ",".join(vars_agrupa)

In [20]:
tendencias(helper, vars_tend, path_tend, params_tend)

----------------------------------------------------------------------------------------------------
                                       0_pmod_tabla_base.sql                                        
----------------------------------------------------------------------------------------------------
    i       tipo                     nombre                     estado     hora_inicio   duracion   
----------------------------------------------------------------------------------------------------
 324/326       DROP             proceso.pmod_tend_tabla_base   finalizado   03:05:08 PM     00:00.3 
 325/326     CREATE             proceso.pmod_tend_tabla_base   finalizado   03:05:08 PM     00:02.2 
 326/326    COMPUTE             proceso.pmod_tend_tabla_base   finalizado   03:05:10 PM     00:01.4 
----------------------------------------------------------------------------------------------------

Resumen de consultas:
-------------------------------------------------
         documento

### Cruce Variables 

In [42]:
helper.ejecutar_archivo(path_etl + "/99_base_modelo/0_base_cruce.sql", params=general_params)

----------------------------------------------------------------------------------------------------
                                          0_base_cruce.sql                                          
----------------------------------------------------------------------------------------------------
    i       tipo                     nombre                     estado     hora_inicio   duracion   
----------------------------------------------------------------------------------------------------
 812/814       DROP                  proceso.pmod_base_cruce   finalizado   04:47:23 PM     00:00.7 
 813/814     CREATE                  proceso.pmod_base_cruce   finalizado   04:47:23 PM     00:02.9 
 814/814    COMPUTE                  proceso.pmod_base_cruce   finalizado   04:47:26 PM     00:01.1 
----------------------------------------------------------------------------------------------------

Resumen de consultas:
-------------------------------------------------
         documento

## DESCARGA BASES

### Saldos diarios

In [26]:
download_query = "SELECT * FROM {proccess_area_zone}.pmod_tend_sd"
data = helper.obtener_dataframe(download_query, params=general_params)
data.head()

----------------------------------------------------------------------------------------------------
    i       tipo                     nombre                     estado     hora_inicio   duracion   
----------------------------------------------------------------------------------------------------
 788/788  DATAFRAME                                             falló n.1   04:11:53 PM             
 788/788  DATAFRAME                                          convirtiendo   04:11:53 PM             

2024-06-23 16:18:13 - [INFO] - 680,800 filas, 88 columnas, 00:21.0 consultando, 05:44.5 descargando, 00:13.6 convirtiendo


 788/788  DATAFRAME                                            finalizado   04:11:53 PM     06:20.0 
----------------------------------------------------------------------------------------------------


,nit_enmascarado,num_oblig_enmascarado,num_oblig_orig_enmascarado,fecha_var_rpta_alt,avg_sld_cap_final_1,avg_sld_cap_final_avg_ult3,avg_sld_cap_final_avg_ult6,avg_sld_cap_final_stddev_ult3,avg_sld_cap_final_stddev_ult6,avg_sld_cap_final_max_ult6,...,max_vlr_vencido_stddev_ult6,max_vlr_vencido_max_ult6,max_vlr_vencido_max_ult3,min_vlr_vencido_1,min_vlr_vencido_avg_ult3,min_vlr_vencido_avg_ult6,min_vlr_vencido_stddev_ult3,min_vlr_vencido_stddev_ult6,min_vlr_vencido_max_ult6,min_vlr_vencido_max_ult3
0,54787,17230,1063112,202308,9396897.16,9.396905e+06,9.396921e+06,1.126809e+01,1.744234e+01,9396936.16,...,7.206160e+03,296389,296389,51,5043.666667,2.521833e+03,7096.789900,5.616214e+03,15080,15080
1,146602,672802,407118,202401,2949134.31,3.001168e+06,3.055246e+06,4.061146e+04,7.162829e+04,3182493.22,...,1.215499e+05,551137,551137,251137,194676.000000,1.199372e+05,79847.911945,9.638677e+04,251137,251137
2,449080,855660,227160,202401,6899545.56,6.899546e+06,5.831331e+06,0.000000e+00,1.682165e+06,6899545.56,...,3.677819e+05,981724,981724,579827,257139.000000,1.285695e+05,241208.541907,2.135905e+05,579827,579827
3,497069,248136,832206,202311,36562047.76,3.656205e+07,3.051209e+07,0.000000e+00,1.364557e+07,36735052.76,...,8.788546e+05,2283824,2283824,1521234,760377.000000,3.801885e+05,621041.272643,5.808523e+05,1521234,1521234
4,4291,453589,626753,202309,2034900.00,3.154857e+06,4.533265e+06,1.583858e+06,1.776038e+06,5911673.00,...,1.343002e+06,3026395,3026395,0,0.000000,1.405516e+06,0.000000,1.407565e+06,2914138,0


In [27]:
data.to_csv('../data/pmod_tend_sd.csv', index=False)

### Maestro cuotas

In [28]:
download_query = "SELECT * FROM {proccess_area_zone}.pmod_tend_mc"
data = helper.obtener_dataframe(download_query, params=general_params)
data.head()

----------------------------------------------------------------------------------------------------
    i       tipo                     nombre                     estado     hora_inicio   duracion   
----------------------------------------------------------------------------------------------------
 789/789  DATAFRAME                                          convirtiendo   04:19:30 PM             

2024-06-23 16:21:57 - [INFO] - 680,800 filas, 25 columnas, 00:00.8 consultando, 02:24.0 descargando, 00:02.2 convirtiendo


 789/789  DATAFRAME                                            finalizado   04:19:30 PM     02:27.3 
----------------------------------------------------------------------------------------------------


,nit_enmascarado,num_oblig_enmascarado,num_oblig_orig_enmascarado,fecha_var_rpta_alt,valor_cuota_mes_1,valor_cuota_mes_avg_ult3,valor_cuota_mes_avg_ult6,valor_cuota_mes_stddev_ult3,valor_cuota_mes_stddev_ult6,valor_cuota_mes_max_ult6,...,pago_total_stddev_ult6,pago_total_max_ult6,pago_total_max_ult3,porc_pago_1,porc_pago_avg_ult3,porc_pago_avg_ult6,porc_pago_stddev_ult3,porc_pago_stddev_ult6,porc_pago_max_ult6,porc_pago_max_ult3
0,408272,653773,426206,202309,249579.0,315162.333333,345228.500000,101782.628830,130351.113387,564603.0,...,824580.668833,2258236.0,2258236.0,0.0,167.666667,134.666667,229.382262,194.250411,492.0,492.0
1,114350,10571,1069771,202401,188172.0,188760.333333,188580.833333,416.419927,489.414077,189077.0,...,401619.234703,1129002.0,377006.0,0.0,66.333333,166.333333,93.809500,213.073279,599.0,199.0
2,606576,482099,598243,202309,30452.0,40218.666667,40308.000000,13994.272217,14169.877958,60681.0,...,120198.248039,361340.0,361340.0,400.0,334.000000,337.333333,250.157284,241.815127,602.0,602.0
3,280731,527381,552961,202312,80151.0,53364.000000,48877.333333,21822.669085,18386.208705,80151.0,...,26652.737237,53428.0,53244.0,0.0,33.333333,50.000000,47.140452,50.000000,100.0,100.0
4,41427,465205,615137,202309,257296.0,257300.000000,257304.166667,5.656854,9.026196,257320.0,...,528950.743348,1560000.0,1560000.0,0.0,269.333333,286.166667,251.938264,205.477020,606.0,606.0


In [29]:
data.to_csv('../data/pmod_tend_mc.csv', index=False)

### Master

In [34]:
download_query = "SELECT * FROM {proccess_area_zone}.pmod_tend_mcd"
data = helper.obtener_dataframe(download_query, params=general_params)
data.head()

----------------------------------------------------------------------------------------------------
    i       tipo                     nombre                     estado     hora_inicio   duracion   
----------------------------------------------------------------------------------------------------
 805/805  DATAFRAME                                          convirtiendo   04:25:58 PM             

2024-06-23 16:30:03 - [INFO] - 680,800 filas, 32 columnas, 00:00.9 consultando, 03:57.2 descargando, 00:06.5 convirtiendo


 805/805  DATAFRAME                                            finalizado   04:25:58 PM     04:05.4 
----------------------------------------------------------------------------------------------------


,nit_enmascarado,num_oblig_enmascarado,num_oblig_orig_enmascarado,fecha_var_rpta_alt,total_ing_1,total_ing_avg_ult3,total_ing_avg_ult6,total_ing_stddev_ult3,total_ing_stddev_ult6,total_ing_max_ult6,...,egresos_mes_stddev_ult6,egresos_mes_max_ult6,egresos_mes_max_ult3,tot_patrimonio_1,tot_patrimonio_avg_ult3,tot_patrimonio_avg_ult6,tot_patrimonio_stddev_ult3,tot_patrimonio_stddev_ult6,tot_patrimonio_max_ult6,tot_patrimonio_max_ult3
0,167120,117849,962493,202311,1010000.00,1010000.000000,673333.333333,0.0,4.761186e+05,1010000.00,...,94280.904158,200000.00,200000.00,4189000.00,4189000.000000,2792666.666666,0.000000,1.974714e+06,4189000.00,4189000.00
1,365996,752619,26615,202401,17000000.00,17000000.000000,17000000.000000,0.0,0.000000e+00,17000000.00,...,0.000000,8000000.00,8000000.00,70000000.00,70000000.000000,70000000.000000,0.000000,0.000000e+00,70000000.00,70000000.00
2,579097,944907,143196,202309,0,0,0,0.0,0.000000e+00,0,...,721248.916810,1530000.00,1530000.00,1174000.00,782666.666666,391333.333333,553428.907409,5.534289e+05,1174000.00,1174000.00
3,596992,169088,911254,202311,3000000.00,3000000.000000,2000000.000000,0.0,1.414214e+06,3000000.00,...,0.000000,0,0,60000000.00,60000000.000000,40000000.000000,0.000000,2.828427e+07,60000000.00,60000000.00
4,311971,40897,1039445,202401,318760627.00,318760627.000000,318760627.000000,0.0,0.000000e+00,318760627.00,...,0.000000,11000000.00,11000000.00,1262495593.00,1262495593.000000,1262495593.000000,0.000000,0.000000e+00,1262495593.00,1262495593.00


In [35]:
data.to_csv('../data/pmod_tend_mcd.csv', index=False)

### PD

In [36]:
download_query = "SELECT * FROM {proccess_area_zone}.pmod_tend_pd"
data = helper.obtener_dataframe(download_query, params=general_params)
data.head()

----------------------------------------------------------------------------------------------------
    i       tipo                     nombre                     estado     hora_inicio   duracion   
----------------------------------------------------------------------------------------------------
 806/806  DATAFRAME                                          convirtiendo   04:30:15 PM             

2024-06-23 16:32:34 - [INFO] - 680,800 filas, 25 columnas, 00:00.7 consultando, 02:15.6 descargando, 00:03.1 convirtiendo


 806/806  DATAFRAME                                            finalizado   04:30:15 PM     02:19.9 
----------------------------------------------------------------------------------------------------


,nit_enmascarado,num_oblig_enmascarado,num_oblig_orig_enmascarado,fecha_var_rpta_alt,prob_propension_1,prob_propension_avg_ult3,prob_propension_avg_ult6,prob_propension_stddev_ult3,prob_propension_stddev_ult6,prob_propension_max_ult6,...,prob_alrt_temprana_stddev_ult6,prob_alrt_temprana_max_ult6,prob_alrt_temprana_max_ult3,prob_auto_cura_1,prob_auto_cura_avg_ult3,prob_auto_cura_avg_ult6,prob_auto_cura_stddev_ult3,prob_auto_cura_stddev_ult6,prob_auto_cura_max_ult6,prob_auto_cura_max_ult3
0,544346,1027226,77860,202311,0.733868,0.777287,0.789446,0.032168,0.031598,0.835386,...,0.108317,0.471343,0.471343,0.443394,0.691520,0.762811,0.175464,0.143347,0.850555,0.818133
1,194619,258342,822000,202312,0.846703,0.861044,0.849784,0.012593,0.020426,0.877358,...,0.038270,0.395424,0.313214,0.723280,0.789793,0.824652,0.055878,0.054328,0.876294,0.860002
2,386545,646176,433978,202401,0.747076,0.768243,0.773170,0.030349,0.024214,0.811162,...,0.071927,0.486903,0.486903,0.376739,0.537999,0.576964,0.127896,0.110928,0.700173,0.689568
3,510084,854406,228417,202308,0.766875,0.778772,0.782536,0.019211,0.017822,0.805873,...,0.054062,0.543687,0.543687,0.409275,0.426047,0.392444,0.024984,0.038157,0.461365,0.461365
4,256444,567399,512943,202312,0.766753,0.858910,0.889944,0.065167,0.055735,0.926652,...,0.163631,0.699793,0.699793,0.307203,0.371780,0.585487,0.055781,0.230931,0.878393,0.443307


In [37]:
data.to_csv('../data/pmod_tend_pd.csv', index=False)

### Variable Respuesta

In [43]:
download_query = "SELECT * FROM {proccess_zone}.pmod_base_cruce"
data = helper.obtener_dataframe(download_query, params=general_params)
data.head()

----------------------------------------------------------------------------------------------------
    i       tipo                     nombre                     estado     hora_inicio   duracion   
----------------------------------------------------------------------------------------------------
 815/815  DATAFRAME                                          convirtiendo   04:48:14 PM             

2024-06-23 16:49:46 - [INFO] - 680,800 filas, 6 columnas, 00:00.7 consultando, 01:30.0 descargando, 00:00.7 convirtiendo


 815/815  DATAFRAME                                            finalizado   04:48:14 PM     01:31.7 
----------------------------------------------------------------------------------------------------


,nit_enmascarado,num_oblig_enmascarado,num_oblig_orig_enmascarado,fecha_var_rpta_alt,var_rpta_alt,marca_pago
0,188431,915475,172650,202401,NaN,PAGO_MENOS
1,128408,238252,842090,202310,1.0,PAGO_MENOS
2,85913,154147,926195,202311,1.0,NO_PAGO
3,156666,665292,414641,202310,1.0,IGUAL
4,282457,594785,485557,202312,1.0,NO_PAGO


In [44]:
data.to_csv('../data/pmod_base_cruce.csv', index=False)